In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("clean_data.csv")
df.head()

,year,month,stateDescription,sectorName,customers,price,revenue,sales
0,2001,1,Wyoming,all sectors,NaN,4.31,48.12840,1116.17208
1,2001,1,Wyoming,commercial,NaN,5.13,12.67978,247.08691
2,2001,1,Wyoming,industrial,NaN,3.26,19.60858,602.30484
3,2001,1,Wyoming,other,NaN,4.75,0.76868,16.17442
4,2001,1,Wyoming,residential,NaN,6.01,15.07136,250.60591


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85870 entries, 0 to 85869
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              85870 non-null  int64  
 1   month             85870 non-null  int64  
 2   stateDescription  85870 non-null  object 
 3   sectorName        85870 non-null  object 
 4   customers         59830 non-null  float64
 5   price             85870 non-null  float64
 6   revenue           85870 non-null  float64
 7   sales             85870 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 5.2+ MB


In [6]:
df.shape

(85870, 8)

In [7]:
df.describe()

,year,month,customers,price,revenue,sales
count,85870.000000,85870.000000,5.983000e+04,85870.000000,85870.000000,85870.000000
mean,2012.043321,6.480144,2.916013e+06,9.300193,586.627155,5980.048970
std,6.660304,3.461589,1.200567e+07,5.010382,2161.047702,21302.453181
min,2001.000000,1.000000,0.000000e+00,0.000000,-0.000010,0.000000
25%,2006.000000,3.000000,4.998000e+03,6.650000,29.475195,289.144572
50%,2012.000000,6.000000,2.997540e+05,8.840000,121.641500,1447.518085
75%,2018.000000,9.000000,2.028716e+06,11.380000,421.320628,4339.950965
max,2024.000000,12.000000,1.625050e+08,116.670000,52361.450970,391900.008970


In [11]:
df.nunique()

year                   24
month                  12
stateDescription       62
sectorName              6
customers           45038
price                2943
revenue             78693
sales               79292
dtype: int64

In [13]:
df.isnull().sum()

year                    0
month                   0
stateDescription        0
sectorName              0
customers           26040
price                   0
revenue                 0
sales                   0
dtype: int64

In [18]:
df.isna().sum() / df.shape[0]

year                0.000000
month               0.000000
stateDescription    0.000000
sectorName          0.000000
customers           0.303249
price               0.000000
revenue             0.000000
sales               0.000000
dtype: float64

In [28]:
df["customers"].fillna(df["customers"].mean(), inplace=True)

In [29]:
df.head()

,year,month,stateDescription,sectorName,customers,price,revenue,sales
0,2001,1,Wyoming,all sectors,2.916013e+06,4.31,48.12840,1116.17208
1,2001,1,Wyoming,commercial,2.916013e+06,5.13,12.67978,247.08691
2,2001,1,Wyoming,industrial,2.916013e+06,3.26,19.60858,602.30484
3,2001,1,Wyoming,other,2.916013e+06,4.75,0.76868,16.17442
4,2001,1,Wyoming,residential,2.916013e+06,6.01,15.07136,250.60591


In [36]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [34]:
categorical_features = ["stateDescription", "sectorName"]
numeric_features = ["year", "month", "customers", "sales"]

In [51]:
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("num", StandardScaler(), numeric_features)
    ], remainder='passthrough')

In [52]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

In [53]:
model = make_pipeline(preprocessor, LinearRegression())

In [46]:
X = df.drop(["price", "revenue"], axis=1)
y = df["price"]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [54]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['stateDescription',
                                                   'sectorName']),
                                                 ('num', StandardScaler(),
                                                  ['year', 'month', 'customers',
                                                   'sales'])])),
                ('linearregression', LinearRegression())])

In [55]:
model.score(X_train, y_train)

0.6457652059670786

In [56]:
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error, r2_score

In [57]:
y_pred = model.predict(X_test)

In [58]:
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R²:", r2_score(y_test, y_pred))

RMSE: 2.961273146432549
R²: 0.6515886984104433


In [61]:
models = {
    "lasso": Lasso(),
    "ridge": Ridge(),
    "elastic": ElasticNet()
}

for k, v in models.items():
    k = make_pipeline(preprocessor, v)
    k.fit(X_train, y_train)
    pred = k.predict(X_test)
    print("RMSE:", mean_squared_error(y_test, pred, squared=False))
    print("R²:", r2_score(y_test, pred))
    print("_" * 20)

RMSE: 4.945161391069011
R²: 0.02838046714498843
____________________
RMSE: 2.961348358839613
R²: 0.6515709998165419
____________________
RMSE: 4.844932567985107
R²: 0.0673670155904148
____________________


In [62]:
import joblib
import pickle

In [66]:
pickle.dump(model, open("regmodel.pkl", "wb"))

In [67]:
X_train.head()

,year,month,stateDescription,sectorName,customers,sales
5714,2002,7,Idaho,other,2.916013e+06,31.35882
12737,2004,6,New Hampshire,transportation,2.916013e+06,0.00000
16870,2005,7,Delaware,transportation,2.916013e+06,0.00000
49132,2014,3,Missouri,industrial,1.027200e+04,1426.09446
61668,2017,7,Nevada,industrial,3.618000e+03,1148.51307
